## Importamos librerías

In [1]:
from math import log10
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Descargar las stopwords la primera vez
# nltk.download('stopwords')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
wnl = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /home/camilo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/camilo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/camilo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Definimos funciones

In [2]:
# Función para tokenizar y procesar el texto
def tokenizer(listas):
    """
    Esta función tokeniza y procesa una lista de textos.
    Parámetros:
        listas: Una lista que contiene los textos a procesar.
    Retorno:
        Una lista de tuplas que contiene el índice del texto original y la lista de palabras procesadas.
    """
    w_token = []
    i = 0
    for x in listas:
        # Convertir a minúsculas y tokenizar
        word_tokens = word_tokenize(x.lower())
        # Eliminar palabras comunes en inglés
        english_stopwords = stopwords.words('english')
        tokens_wo_stopwords = [t for t in word_tokens if t not in english_stopwords]
        # Lematizar las palabras
        lemma = [wnl.lemmatize(t) for t in tokens_wo_stopwords]
        # Crear tupla con índice y lista de palabras
        tupla = (i, lemma)
        w_token.append(tupla)
        i += 1
    return w_token

# Función para calcular la frecuencia normalizada (TF)            
def tf(lista):
    """
    Esta función calcula la frecuencia normalizada (TF) de las palabras en una lista de documentos.
    Parámetros:
        lista: Una lista de tuplas que contiene el índice del documento y la lista de palabras procesadas.
    Retorno:
        Una lista de tuplas que contiene el índice del documento, la palabra y la frecuencia normalizada.
    """
    frec_word_list = []
    for words_tuple in lista:
        # Desempaquetar la tupla
        doc_id, list_of_words = words_tuple
        # Calcular la longitud del documento
        doc_length = len(list_of_words)
        frec = {}
        # Contar la frecuencia de cada palabra      
        for word in list_of_words:
            if word in frec:
                frec[word] += 1
            else:
                frec[word] = 1
        # Calcular la frecuencia normalizada    
        for key, value in frec.items():
            frec_word_list.append(((doc_id,key),value/doc_length))                
    return frec_word_list

# Función para calcular el peso por documento inverso (IDF)
def idf(lista,len_doc_total):
    """
    Esta función calcula el peso por documento inverso (IDF) de las palabras en una lista de documentos.
    Parámetros:
        lista: Una lista de tuplas que contiene el índice del documento y la lista de palabras procesadas.
        len_doc_total: La cantidad total de documentos.
    Retorno:
        Un diccionario que contiene la palabra y su peso IDF.
    """
    frec_word_list = {}
    for i, words in lista:
        for word in words:
            if word in frec_word_list:
                frec_word_list[word] += 1            
            else:
                frec_word_list[word] = 1
    result = {}
    for key, value in frec_word_list.items():
        result[key] = log10(len_doc_total/value)
    return result  

# Función para calcular el TF-IDF
def tf_idf(tf_list, idf_list):
    """
    Esta función calcula el TF-IDF de las palabras en una lista de documentos.
    Parámetros:
        tf_list: Una lista de tuplas que contiene el índice del documento, la palabra y la frecuencia normalizada.
        idf_list: Un diccionario que contiene la palabra y su peso IDF.
    Retorno:
        Una lista de tuplas que contiene el índice del documento, la palabra y el valor TF-IDF.
    """
    tf_idf_list = []
    for (doc_id, word), tf_word in tf_list:
        # Multiplicar TF por IDF        
        if word in idf_list:
            tf_idf_value = idf_list[word] * tf_word
            tf_idf_list.append(((doc_id, word), tf_idf_value))
    return tf_idf_list

# Función principal que lee el archivo txt y entrega los documentos a las funciones anteriormente declaradas
def main(archivo):
    """
    Esta función principal lee un archivo de texto, lo procesa y calcula el TF-IDF de las palabras.
    Parámetros:
        archivo: La ruta al archivo de texto.
    Retorno:
        Una lista de tuplas que contiene el índice del documento, la palabra y el valor TF-IDF.
    """
    with open(archivo, 'r', encoding='utf-8') as file:
        new_lista = tokenizer(file)
        len_doc_total = len(new_lista)
        tf_list = tf(new_lista)
        idf_list = idf(new_lista,len_doc_total)
        tf_idf_list = tf_idf(tf_list,idf_list)        
    return tf_idf_list        

In [3]:
tf_idf = main('pg100.txt')

In [4]:
tf_idf

[((0, '\ufeffthe'), 0.5662410382201679),
 ((0, 'project'), 0.286114584475275),
 ((0, 'gutenberg'), 0.29847069118335745),
 ((0, 'ebook'), 0.42952449140035975),
 ((0, 'complete'), 0.30038582119131624),
 ((0, 'work'), 0.26464884269291983),
 ((0, 'william'), 0.283018453550636),
 ((0, 'shakespeare'), 0.2957333822163681),
 ((0, ','), 0.019495174534766922),
 ((1, 'william'), 1.273583040977862),
 ((1, 'shakespeare'), 1.3308002199736564),
 ((3, 'ebook'), 0.7731440845206476),
 ((3, 'use'), 0.46990702900878656),
 ((3, 'anyone'), 0.850214260793451),
 ((3, 'anywhere'), 0.8386158713979136),
 ((3, 'cost'), 0.668058897661804),
 ((4, 'almost'), 0.3207393699376534),
 ((4, 'restriction'), 0.53279326092417),
 ((4, 'whatsoever'), 0.4355642316584256),
 ((4, '.'), 0.023162449305868863),
 ((4, 'may'), 0.20249708400571326),
 ((4, 'copy'), 0.2682249186230755),
 ((4, ','), 0.019495174534766922),
 ((4, 'give'), 0.21402235222051322),
 ((4, 'away'), 0.24035428022870148),
 ((5, 're-use'), 0.799189891386255),
 ((5, '